# API: Medal count & rank
This API gets the current Top 10 rank standings of the Olympics 2024 medals. 

In [1]:
#API Summary: To get the top 10 ranked countries with medal counts in Paris 2024 Olympics.

from googletrans import Translator

import requests
import json

translator = Translator()

rename_dict = "Great Britain" # When using the translator, this only outputs 'Briton' so as a workaround I use a variable replacement instead shown later in the code.


url = "https://apis.codante.io/olympic-games/countries" # Base URL for the API.
response = requests.get(url)

if response.status_code == 200: # Status code 200 is that it is able to connect successfully to the API and can collect the API data.
  data = response.json()

  countries = data['data'] #accessing API data dictionary.


  for i, country in enumerate(countries[:10]): # Loop through the top 10 ranked countries of 2014 Olympics and print the medal information
    original_name = country['name']
    translated_name = translator.translate(original_name, src='pt', dest='en').text # I am using a translator as the API end points are in Portuguese, this takes in the original name on the API, the source language is Portuguese and the destination output is returned in English.
#Getting the medal information for gold, silver, bronze and total medals alongside where they ranked in the 2024 olympic games.
    gold_medals = country['gold_medals']
    silver_medals = country['silver_medals']
    bronze_medals = country['bronze_medals']
    total_medals = country['total_medals']
    rank = country['rank']
    print(f"Rank: {rank}")
    if country['name'] == "Grã-Bretanha": #As commented earlier in the code; using a if statement to replace this to Great Britain, otherwise portuguese output is 'Briton' without this.
      print(f"Country: {rename_dict}")
    else:
      print(f"Country: {translated_name}") # goes back to using the translated data.
    print(f"Gold Medals: {gold_medals}")
    print(f"Silver Medals: {silver_medals}")
    print(f"Bronze Medals: {bronze_medals}")
    print(f"Total Medals: {total_medals}")
    print("-" * 10)  # used a separator for better readability
else:
  print(f"Failed to retrieve data. Status code: {response.status_code}")



Rank: 1
Country: USA
Gold Medals: 40
Silver Medals: 44
Bronze Medals: 42
Total Medals: 126
----------
Rank: 2
Country: China
Gold Medals: 40
Silver Medals: 27
Bronze Medals: 24
Total Medals: 91
----------
Rank: 3
Country: Japan
Gold Medals: 20
Silver Medals: 12
Bronze Medals: 13
Total Medals: 45
----------
Rank: 4
Country: Australia
Gold Medals: 18
Silver Medals: 19
Bronze Medals: 16
Total Medals: 53
----------
Rank: 5
Country: France
Gold Medals: 16
Silver Medals: 26
Bronze Medals: 22
Total Medals: 64
----------
Rank: 6
Country: Netherlands
Gold Medals: 15
Silver Medals: 7
Bronze Medals: 12
Total Medals: 34
----------
Rank: 7
Country: Great Britain
Gold Medals: 14
Silver Medals: 22
Bronze Medals: 29
Total Medals: 65
----------
Rank: 8
Country: Republic of Korea
Gold Medals: 13
Silver Medals: 9
Bronze Medals: 10
Total Medals: 32
----------
Rank: 9
Country: Italy
Gold Medals: 12
Silver Medals: 13
Bronze Medals: 15
Total Medals: 40
----------
Rank: 10
Country: Germany
Gold Medals: 12
Sil

# NOTE
Before proceeding on, please make sure you do the following:
- Have the olympics database alongside the medal_events table
- Have a config file which has your SQL credentials to connect to the database.

# Using the external API, connecting to our SQL database and inserting Team GB data
The collected data from the API is the date of the event took place, the sport and the medal won.
Keep patient, this can take some time to run as there's over 4000 olympic game entries to go through!

<b>API Summary:</b> This API was getting all the sporting events in Paris 2024 Olympics API that Team GB won a medal.
- Connects with an external API, gets the API data and updates our SQL database with the date of event, the sport and medal
- This can then be viewed on our own API server using the app.py file.

In [ ]:
import requests
import mysql.connector
from datetime import datetime
from config import USER, PASSWORD, HOST # config file holds SQL credentials

#Database connection parameters

HOST = HOST
USER = USER
PASSWORD = PASSWORD
DATABASE = 'olympics' # connect to the olympics database

# API URL end point to get the event details: date of event, sport, medal

url = 'https://apis.codante.io/olympic-games/events' 

# Function to process and insert event data into the database
def find_insert(event, cursor):
    sport_name = event.get('discipline_name', 'Unknown') # using parameter 'unknown' and 'n/a' to avoid getting errors for missing data within the API.
    # So instead of giving an error, it will provide this parameter output as a replacement instead to continue running though the API entries.
    event_date = event.get('day', 'Unknown')

    # Have the date format as YYYY-MM-DD for SQL
    if event_date != 'Unknown':
        event_date = datetime.strptime(event_date, '%Y-%m-%d').strftime('%Y-%m-%d')

    # Checks medal event = 1 as that means it was a final where a medal was provided, whereas medal_event equalling 0 is not.
    if event.get('is_medal_event') == 1:
        for competitor in event.get('competitors',[]):
            country_id = competitor.get('country_id', 'Unknown')
            if country_id == "Grã-Bretanha": # as the API is in Portuguese, using this term which translates to Great Britain.
                result_position = competitor.get('result_position', 'N/A') # result being first, second or third placed in the event.

    #converting the winning results of 1,2,3 into relevant medals using else if statements if no medals found, continue with code.
                if result_position == '1':
                    medal = 'Gold'
                elif result_position == '2':
                    medal = 'Silver'
                elif result_position == '3':
                    medal = 'Bronze'
                else:
                    continue # skip the rest of the loop

    # Insert data into the olympics database in the medal_events table

                cursor.execute("""
                    INSERT INTO medal_events (event_date, discipline_name, medal)
                    VALUES (%s, %s, %s)
                """, (event_date, sport_name, medal))

# Function to fetch and update the data

def fetupdate(cursor):
    params = {
        'is_medal_event': 1, # confirms that only medalled events get updated.
        'limit': 100 # The API has a limit of 100 request calls.
    }

# as this API has lots of entries over multiple pages 
# start from page 1 and will go through each of them.
    page = 1 
    while True:
        params['page'] = page
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e: # raising an exception error for any pages that fail to be retrieved.
            print(f"Failed to retrieve page {page}: {e}")
            break

        data = response.json()
        if not data.get('data'):
            break

        for event in data.get('data', []): # for loop to insert the event data into sql using the find_insert function.
            find_insert(event, cursor)

        page += 1 # continue looping through each page of the API until it ends.

# function to connect to the olympics database
def main():
    try:
        conn = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DATABASE
        )
        cursor = conn.cursor()

        # Fetch and update the database
        fetupdate(cursor)

        # Commit the fetched updates to the database
        conn.commit()

        #raise exception error if needed.

    except mysql.connector.Error as err:
        print(f"Database error: {err}")

    # Close the connection and cursor
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    main()


        
sor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    main()


        


# Using Flask App
Once we have gathered the data from the external source and inserted into our SQL database, we can then use Flask web application to use our own server API.  

In [ ]:
# Summary: Connecting to our SQL database and viewing the Team GB 2024 Sports from our own server API.

import mysql.connector
from mysql.connector import Error
from flask import Flask, jsonify
from config import USER, PASSWORD, HOST

app = Flask(__name__)

try:
    # Establishing connection to the database
    conn = mysql.connector.connect(
        host=HOST,
        user=USER,
        password = PASSWORD,
        database='olympics'
    )

    # Create a cursor object using the connection
    cur = conn.cursor()
    cur.execute("SELECT * FROM medal_events")

    # Fetch all results from the executed query

    data = cur.fetchall()

    # Get the column names from the cursor description
    columns = [col[0] for col in cur.description]

    # Initialising an empty list to hold the data dictionaries.

    data_dict = []

    # Looping through each row in the data
    for row in data:
        row_dict = {} # creates a dictionary for each row
        # Loop through each column and corresponding value in the row
        for i in range(len(columns)):
            row_dict[columns[i]] = row[i] # Mapping the column name to the relevant row 
        # Add the dictionary to the list
        data_dict.append(row_dict)

except mysql.connector.Error as err:
    print(f"Error: {err}") # Provides us any errors that we run into.

# Once we are done, we close the cursor and connection for security purposes
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

#Creates our landing page
@app.route('/')
def get_landing_page():
    res = {"2024 Olympic Games": "CFG Group Project"}
    return jsonify(res)

# New URL endpoint for Team GB won events.
@app.route('/teamgb/')
def get_data():
    return jsonify(data_dict) # Returns the data in JSON dictionary format.

if __name__ == '__main__':
    app.run(debug=True)




# Wikipedia API
We noticed that we were missing key athletes from our dataset. We came across a API for Wikipedia which helped us gather the missing atheletes into a spreadsheet so we can do another data clean.

API Documentation: https://pypi.org/project/Wikipedia-API/

In [1]:
!pip install wikipedia-api

In [2]:
import wikipediaapi

wiki = wikipediaapi.Wikipedia('OlympicsInfo (youremail@example.com)', 'en')

page_py = wiki.page('List of multiple Olympic medalists')
print("Page - Exists: %s" % page_py.exists()) # Checks Wikipedia that the page exists.


def print_sections(sections, level=0):
    for s in sections:
        print("%s: %s - %s" % ("*" * (level + 1), s.title, s.text[0:40]))
        print_sections(s.sections, level + 1)

print_sections(page_py.sections)

Page - Exists: True
*: List of Olympic medals over career - This list includes athletes who have won
**: Timeline - This is a progressive list of Olympians 
*: List of most career medals in individual events - This list contains only medals won in in
*: Athletes with medals in different disciplines - 
**: In the Summer and Winter Games - Gillis Grafström became the first person
**: In the Summer Games - 
***: Swimming and water polo - Paul Radmilovic (GBR)
 Johnny Weissmulle
***: Others in Summer Games - Morris Kirksey (USA) (athletics and rugb
**: In the Winter Games - 
***: Cross-country skiing and Nordic combined - Thorleif Haug (NOR)
 Johan Grøttumsbråte
***: Others in Winter Games - Anfisa Reztsova (URS) (biathlon and cros
*: See also - List of multiple Olympic medalists at a 
*: References - 
"Olympic Medal Winners". International 


In [3]:
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='MyProjectName (merlin@example.com)',
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("List of multiple Olympic medalists")
print(p_wiki.text)

This article provides a list of multiple Olympic medalists, i.e. those athletes who have won multiple Olympic medals at either the Summer Olympic Games or the Winter Olympic Games.

List of Olympic medals over career
This list includes athletes who have won six or more Olympic medals over their sporting career. It includes top-three placings in the 1896 Olympic Games and 1900 Olympic Games, before medals were actually awarded for those placings. Medals won in the 1906 Intercalated Games are not included.
For simplicity, when an athlete has won medals for more than one nation, their entry in this list only mentions the last Nation represented. The Years listed for each athlete only include the Games in which they won medals. More detailed information is provided in the linked articles for the individual athletes.
In cases where two or more athletes have the same number of total medals, the first tiebreaker is the number of gold medals, followed by the number of silver medals. If the tie